<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo3/blob/main/notebooks/TratandoBaseOriginal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para redução do tamanho da base que será utilizada
## Manteremos os Estados e as cidades com mais de 1,5 milhões de habitantes
### também decidi retirar o último mês (dezembro de 2021) da base, dado o risco de estar incompleto

In [1]:
import pandas as pd # importando o pandas
pd.options.display.float_format = "{:,.2f}".format # definindo formato dos números

lendo os dados extraidos do [Brasil.io](https://brasil.io/dataset/covid19/caso_full/) no dia 30 de dezembro de 2021.

In [2]:
dados = pd.read_csv('/content/caso_full.csv.gz', compression='gzip')

In [48]:
print(dados.shape) # conferindo o tamanho da base
dados.head() # verificando os primeiros dados para entender as informações

(3359440, 18)


,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,"1,200,401.00",2020-03-17,202012,"413,418.00","407,319.00",False,False,3,0.73,2020-03-17,0.00,0,1,city,AC,3,0
1,NaN,12.00,2020-03-17,202012,"894,470.00","881,935.00",False,False,3,0.34,2020-03-17,0.00,0,1,state,AC,3,0
2,Rio Branco,"1,200,401.00",2020-03-18,202012,"413,418.00","407,319.00",False,False,3,0.73,2020-03-18,0.00,0,2,city,AC,0,0
3,NaN,12.00,2020-03-18,202012,"894,470.00","881,935.00",False,False,3,0.34,2020-03-18,0.00,0,2,state,AC,0,0
4,Rio Branco,"1,200,401.00",2020-03-19,202012,"413,418.00","407,319.00",False,False,4,0.97,2020-03-19,0.00,0,3,city,AC,1,0


Para reduzir o tamanho da base decidi manter apenas as maiores cidades do Brasil, escolhi 1,5 milhões de habitantes que por coincidencia deu um número de 10 cidades. 

Decidi também manter a informação consolidada dos estados, para ter uma informação total caso eu queira avaliar.

In [53]:
# criando a lista das cidades com mais do que 1,5 milhão de habitantes
cidades = dados[['city','estimated_population']].query('estimated_population > 1500000').groupby(by='city').mean()
lista_cidades = list(cidades.index.values)
lista_cidades

['Belo Horizonte',
 'Brasília',
 'Curitiba',
 'Fortaleza',
 'Goiânia',
 'Manaus',
 'Recife',
 'Rio de Janeiro',
 'Salvador',
 'São Paulo']

In [60]:
# criando a base filtrando a visão consolidada dos estados e as cidades selecionadas anteriormente
dados_saida = dados.query('place_type == "state" or city in @lista_cidades')
dados_saida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24340 entries, 1 to 3359439
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   city                                           6597 non-null   object 
 1   city_ibge_code                                 24340 non-null  float64
 2   date                                           24340 non-null  object 
 3   epidemiological_week                           24340 non-null  int64  
 4   estimated_population                           24340 non-null  float64
 5   estimated_population_2019                      24340 non-null  float64
 6   is_last                                        24340 non-null  bool   
 7   is_repeated                                    24340 non-null  bool   
 8   last_available_confirmed                       24340 non-null  int64  
 9   last_available_confirmed_per_100k_inhabitants  2

como sei que tivemos problemas com os dados sobre covid no mês de dezembro ([Sem dados de 4 estados, Brasil registra 173 mortes por Covid em 24 horas; sistemas seguem com problema pelo 7º dia após ataque hacker](https://g1.globo.com/saude/coronavirus/noticia/2021/12/16/sem-dados-de-5-estados-brasil-registra-173-mortes-por-covid-em-24-horas-sistemas-seguem-com-problema-uma-semana-apos-ataque-hacker.ghtml) - G1) e também sabemos que os últimos dias costumam ter problemas de informação que costumam ser completadas nos dias seguintes, decidi retirar o mês de dezembor da base. Com isso, a base só terá informações até novembro de 2021.

In [61]:
# transformando o campo 'date' para o formato de data 
dados_saida['date'] = pd.to_datetime(dados_saida["date"])
dados_saida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24340 entries, 1 to 3359439
Data columns (total 18 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   city                                           6597 non-null   object        
 1   city_ibge_code                                 24340 non-null  float64       
 2   date                                           24340 non-null  datetime64[ns]
 3   epidemiological_week                           24340 non-null  int64         
 4   estimated_population                           24340 non-null  float64       
 5   estimated_population_2019                      24340 non-null  float64       
 6   is_last                                        24340 non-null  bool          
 7   is_repeated                                    24340 non-null  bool          
 8   last_available_confirmed                       24340 n

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [72]:
# filtrando as datas, retirando o mês de dezembro
dados_saida = dados_saida.query("date < '2021-12-01'")
print(dados_saida.shape) # conferindo o tamanho final da base

(23267, 18)


In [70]:
dados_saida.to_csv('/content/dados_covid.csv', index=False)